# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Mon Apr 26 13:59:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   48C    P8     8W / 215W |    283MiB /  7974MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.config.list_physical_devices('XLA_GPU'))
device_lib.list_local_devices()

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6507124110760788315,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4922654194336399393
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1179884248341804191
 physical_device_desc: "device: XLA_GPU device"]

In [4]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.6378, 0.9107, 0.5509],
        [0.4454, 0.1930, 0.4130],
        [0.0074, 0.9115, 0.1397],
        [0.4758, 0.0569, 0.8469],
        [0.1666, 0.7414, 0.5519]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

358 ms ± 72.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

187 ms ± 40.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.5021, 0.1432, 0.7876, 0.4729, 0.8067],
        [0.7827, 0.3770, 0.8910, 0.3543, 0.3826],
        [0.1806, 0.2917, 0.3224, 0.2717, 0.3795],
        [0.5002, 0.2753, 0.5238, 0.0830, 0.9391],
        [0.0774, 0.3479, 0.8384, 0.6825, 0.4502]], device='cuda:0')
tensor([[0.5021, 0.1432, 0.7876, 0.4729, 0.8067],
        [0.7827, 0.3770, 0.8910, 0.3543, 0.3826],
        [0.1806, 0.2917, 0.3224, 0.2717, 0.3795],
        [0.5002, 0.2753, 0.5238, 0.0830, 0.9391],
        [0.0774, 0.3479, 0.8384, 0.6825, 0.4502]], dtype=torch.float64)


In [10]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

8.42 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Exhaustive Testing on GPU

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [12]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.6581, 0.1584, 0.1951, 0.7333, 0.2423],
        [0.7057, 0.4649, 0.6851, 0.4686, 0.4990],
        [0.0904, 0.3937, 0.1658, 0.3733, 0.8904],
        [0.2138, 0.0609, 0.5688, 0.1917, 0.8857],
        [0.2110, 0.6726, 0.2961, 0.3625, 0.0745]], device='cuda:0')


In [13]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border. 
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [14]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 7.5589e-04,  9.0142e-05,  1.7263e-04,  7.5191e-05,  1.2231e-04],
        [ 9.0142e-05,  5.8283e-04, -2.2237e-04,  3.5647e-04, -2.3438e-05],
        [ 1.7263e-04, -2.2237e-04,  8.8304e-04,  3.3944e-04, -4.9345e-05],
        [ 7.5191e-05,  3.5647e-04,  3.3944e-04,  9.6286e-04, -1.4842e-05],
        [ 1.2231e-04, -2.3438e-05, -4.9345e-05, -1.4842e-05,  1.0476e-03]],
       device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred 
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 7.5589e-04,  9.0142e-05,  1.7263e-04,  7.5191e-05,  1.2231e-04],
        [ 9.0142e-05,  5.8283e-04, -2.2237e-04,  3.5647e-04, -2.3438e-05],
        [ 1.7263e-04, -2.2237e-04,  8.8304e-04,  3.3944e-04, -4.9345e-05],
        [ 7.5191e-05,  3.5647e-04,  3.3944e-04,  9.6286e-04, -1.4842e-05],
        [ 1.2231e-04, -2.3438e-05, -4.9345e-05, -1.4842e-05,  1.0476e-03]],
       dtype=torch.float64)
